In [1]:
import pandas as pd
import numpy as np

In [2]:
eur = {'EUR','USD','CAD','JPY'}
#df = pd.DataFrame(columns='EUR','USD','CAD','JPY'}
print(eur)
df = pd.DataFrame(eur)
df[0]['EUR']=3
df.head()

{'CAD', 'USD', 'JPY', 'EUR'}


,0
0,CAD
1,USD
2,JPY
3,EUR


In [3]:
a = pd.Series([1,2,3])
print(a)

0    1
1    2
2    3
dtype: int64


In [4]:
df_events = pd.DataFrame(index=['EUR','USD','AUD']) #rows of df
df_events.loc[:,'Mar23']=pd.Series([1,2,3],index=df_events.index) #adds new column
df_events

,Mar23
EUR,1
USD,2
AUD,3


In [5]:
df_events = pd.DataFrame(index=['EUR','USD','AUD'],columns=[1,2,3]) #rows of df
#df_events.loc[:,'Mar23']=pd.Series([1,2,3],index=df_events.index)
#df_events.loc[:,'Mar24']=pd.Series([2,3,4],index=df_events.index)
df_events

,1,2,3
EUR,NaN,NaN,NaN
USD,NaN,NaN,NaN
AUD,NaN,NaN,NaN


Fill in hr columns

In [6]:
df_events = pd.DataFrame(index=['Mar23.2020','Mar24.2020','Mar25.2020'],columns=pd.Series(range(0,24))) #rows of df
df_events.loc['Mar23.2020'][0]={'USD','EUR','CAD'}
df_events

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Mar23.2020,"{CAD, USD, EUR}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mar24.2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mar25.2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_test = pd.DataFrame(index=['Mar23.2020','Mar24.2020','Mar25.2020'],columns=pd.Series(range(0,3))) #rows of df
my_str='USD'
df_test.at['Mar23.2020',0]=my_str
print( df_test )
my_str=my_str+',CAD'
df_test.at['Mar23.2020',0]=my_str
print(df_test)
my_str=my_str+','+'NZD'
df_test.at['Mar23.2020',0]=my_str
print(df_test)

              0    1    2
Mar23.2020  USD  NaN  NaN
Mar24.2020  NaN  NaN  NaN
Mar25.2020  NaN  NaN  NaN
                  0    1    2
Mar23.2020  USD,CAD  NaN  NaN
Mar24.2020      NaN  NaN  NaN
Mar25.2020      NaN  NaN  NaN
                      0    1    2
Mar23.2020  USD,CAD,NZD  NaN  NaN
Mar24.2020          NaN  NaN  NaN
Mar25.2020          NaN  NaN  NaN


In [8]:
#is it relatively easy to loop over this type of data?
for ii in str.split(df_test.loc['Mar23.2020',0],','):
    print(ii)

USD
CAD
NZD


In [9]:
if( pd.isna(df_test.loc['Mar24.2020',0]) ):
    print("no element yet, NaN")
else:
    for ii in str.split(df_test.loc['Mar24.2020',0],','):
        print(ii)

no element yet, NaN


Set up a loop and make all the columns

In [12]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://www.forexfactory.com/calendar.php?day=Mar22.2020") #sample page
content = page.content
soup = BeautifulSoup(content,"html.parser")
table = soup.find_all("tr",{"class":"calendar_row"})
time=99
for item in table:
    currency=item.find_all("td", {"class":"calendar__currency"})[0].text #currency
    time_entry=item.find_all("td", {"class":"calendar__time"})[0].text #time (may have more than one entry)
    #if( str.len(time_entry)=0 ):  #empty string for time
    #        time_entry=time
    
    print( "currency="+currency ) #Currency
    print( "time_entry=",time_entry ) #Time Eastern

TooManyRedirects: Exceeded 30 redirects.

In [ ]:
currency.strip('\n')

In [ ]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://www.forexfactory.com/calendar.php?day=Mar25.2020") #sample page
content = page.content
soup = BeautifulSoup(content,"html.parser")
table = soup.find_all("tr",{"class":"calendar_row"})
time=99
for item in table[0:2]:
#for item in table:
    currency=item.find_all("td", {"class":"calendar__currency"})[0].text #currency
    currency=currency.strip('\n') #removes both front and back \n 
    time_entry=item.find_all("td", {"class":"calendar__time"})[0].text #time (may have more than one entry)
    if( len(time_entry)==0 ):  #empty string for time
        time_entry=time #previous time
        #print("boom0")
    time=time_entry
    print("currency=",currency)
    #print( "currency="+currency ) #Currency
    print( "time_entry=",time_entry ) #Time Eastern

we'll need a function to translate time into 24hr cycle

In [ ]:
print(type(time_entry))

In [ ]:
def conv_ampm_to_twentyFourHrString(string1):
    """Converts 12hr time to 24hr time, e.g. 11:00pm-->23:00,11:00am-->11:00"""
    conv_t=0
    if( "am" in string1  ): #am
        conv_t=string1[0:len(string1)-2]
    elif( "pm" in string1  ): #pm
        starthr=int(string1.split(':')[0])
        if(starthr==12):
            conv_t=string1.strip('pm') #grab 12:00pm as 12:00
        else:
            conv_t=str(starthr+12)+":"+string1.split(':')[1]#+12hr for any other pm time
            conv_t=conv_t.strip('pm')
    else:  #
        raise ValueError('am or pm time not provided to conv_to_twentyFourHr')
    return conv_t

In [ ]:
print( conv_ampm_to_twentyFourHrString("12:00am") )

Perform our analysis 

In [ ]:
curr = []
times = []
for item in table:
    currency=item.find_all("td", {"class":"calendar__currency"})[0].text #currency
    currency=currency.strip('\n') #removes both front and back \n 
    time_entry=item.find_all("td", {"class":"calendar__time"})[0].text #time (may have more than one entry)
    if( len(time_entry)==0 ):  #empty string for time
        time_entry=time #previous time
    else:
        time=conv_ampm_to_twentyFourHrString(time_entry)
    curr.append(currency)
    times.append(time)

In [ ]:
df_cal = pd.DataFrame([times,curr])
print(df_cal)
df_cal = df_cal.transpose()
print(df_cal)

In [ ]:
print(df_cal.iloc[0,0])
print(df_cal.iloc[1,0])

In [ ]:
df_events = pd.DataFrame(index=['Mar23.2020','Mar24.2020','Mar25.2020'],columns=pd.Series(range(0,24))) #rows of df
print(df_events)

In [ ]:
for ii in df_cal.iloc[:,0]: #loop over times column in df
    print(ii)
    hr_rd=int(ii.split(':')[0])
    print(hr_rd)

In [ ]:
print(df_cal)

In [ ]:
print(df_cal.iloc[3][1])
print(type(df_events.at['Mar25.2020',3]))

In [ ]:
df_test = pd.DataFrame(index=['Mar23.2020','Mar24.2020','Mar25.2020'],columns=pd.Series(range(0,24)))
print(df_events)
print( pd.isna(df_events.at['Mar25.2020',2]) )

Lets define a function to access the dateframe at a particular hour and spit out the currencies with an event

In [ ]:
def check_econCal_curr(check_df,check_date,check_hr):
    """Check dataframe of ecnomic calendar events for relevant currencies. \n
       Returns a list of currencies at specified check_date,check_hr from check_df"""
    thecurr = []
    if check_date in check_df.index: #if index exists
        if( pd.isna(check_df.loc[check_date,check_hr])==False ):#access index,time for event
            for cc in str.split(check_df.loc[check_date,check_hr],','):
                thecurr.append(cc)
    else:
        raise ValueError('The dataframe does not have an index '+check_date)
    return thecurr

In [ ]:
a=check_econCal_curr(df_test,'Mar23.2020',0)
print(a)
print(type(a))

In [ ]:
?check_econCal_curr

In [ ]:
m_table = pd.Series( {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'June':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12} )
#m_strTable
print(m_table)
m_table['Jan']

In [ ]:
m_table = pd.Series( {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'} )
#m_strTable
print(m_table)
print(m_table[2][:])#accesses month string
a=m_table[3][:]+'23.2020'
print(a)

## function / library development

Later on we will want to access a range of calendar dates.  Lets see what type of objects this will entale before we define our dataframe-building functions

In [ ]:
import calendar
num_days = calendar.monthrange(2020,1)[1] #gives total number of days in the month
print(num_days) 

In [ ]:
def monthInt_toStr(m_int):
    """write string function to convert integer to month string for webpage"""
    m_string = ""
    if( isinstance(m_int,int) and m_int>=1 and m_int<=12 ):# of integer type
        m_table = pd.Series( {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'} )
        m_string = m_table[m_int][:]# three-letter month string        
    else:
        raise ValueError("month must be an integer of value 1-12")
    return m_string    

In [ ]:
def webscrape_forexFactory_page(websc_month,websc_day,websc_year):
    """Scrape www.forexfactory.com/calendar for currency events for a particular day"""
    s=requests.Session()
    s.max_redirects = 60
    s.headers = {'User-Agent': 'My App'}
    websc = monthInt_toStr(websc_month)+str(websc_day)+ '.' +str(websc_year)
    webstring="https://www.forexfactory.com/calendar.php?day="+websc
    print("webstring=",webstring)
    page = s.get(webstring) #page scraping
    content = page.content
    soup = BeautifulSoup(content,"html.parser")
    table = soup.find_all("tr",{"class":"calendar_row"})

    curr = []
    times = []
    timeString=""
    for item in table:
        currency=item.find_all("td", {"class":"calendar__currency"})[0].text #currency
        currency=currency.strip('\n') #removes both front and back \n 
        currency=currency.replace(" ","")
        time_entry=item.find_all("td", {"class":"calendar__time"})[0].text #time (may have more than one entry)
        #print("time_entry=",time_entry)
        #print("currency=",currency)
        #print("len(currency)=",len(currency))
        #print("len(time_entry)=",len(time_entry))
        if( time_entry=='All Day' ):
            time_entry='00:00' #All day events at start of day
        elif( len(time_entry)==0 and len(currency)>0 ):  #empty string for time
            time_entry=timeString #previous time
        elif( len(time_entry)>0 and len(currency)>0 ):
            time_entry=conv_ampm_to_twentyFourHrString(time_entry)
            timeString=time_entry
        else:
            return #returns none, no events for that day
        curr.append(currency)
        times.append(time_entry)  
    #print("final times=",times)
    #print("final curr=",curr)
    return pd.DataFrame([times,curr]).transpose()

In [ ]:
#def webscrape_forexFactory_page(websc_month,websc_day,websc_year):
    """Scrape www.forexfactory.com/calendar for currency events for a particular day"""
    
    websc = monthInt_toStr(websc_month)+str(websc_day)+ '.' +str(websc_year)
    page = requests.get("https://www.forexfactory.com/calendar.php?day="+websc) #page scraping
    content = page.content
    soup = BeautifulSoup(content,"html.parser")
    table = soup.find_all("tr",{"class":"calendar_row"})

    curr = []
    times = []
    timeString=""
    for item in table:
        currency=item.find_all("td", {"class":"calendar__currency"})[0].text #currency
        currency=currency.strip('\n') #removes both front and back \n 
        currency=currency.replace(" ","")
        time_entry=item.find_all("td", {"class":"calendar__time"})[0].text #time (may have more than one entry)
        #print("time_entry=",time_entry)
        #print("currency=",currency)
        #print("len(currency)=",len(currency))
        #print("len(time_entry)=",len(time_entry))
        if( time_entry=='All Day' ):
            time_entry='00:00' #All day events at start of day
        elif( len(time_entry)==0 and len(currency)>0 ):  #empty string for time
            time_entry=timeString #previous time
        elif( len(time_entry)>0 and len(currency)>0 ):
            time_entry=conv_ampm_to_twentyFourHrString(time_entry)
            timeString=time_entry
        else:
            return #returns none, no events for that day
        curr.append(currency)
        times.append(time_entry)  
    #print("final times=",times)
    #print("final curr=",curr)
    return pd.DataFrame([times,curr]).transpose()

In [ ]:
test_cal=pd.DataFrame()
test_cal=webscrape_forexFactory_page(2,5,2018)
print(test_cal)
print(type(test_cal))

In [ ]:
df_cal = pd.DataFrame([times,curr])
print(df_cal)
df_cal = df_cal.transpose()
print(df_cal)

In [ ]:
         time=99
    for item in table:
        currency=item.find_all("td", {"class":"calendar__currency"})[0].text #currency
        time_entry=item.find_all("td", {"class":"calendar__time"})[0].text #time (may have more than one entry)
        #if( str.len(time_entry)=0 ):  #empty string for time
        #        time_entry=time

        print( "currency="+currency ) #Currency
        print( "time_entry=",time_entry ) #Time Eastern 

In [ ]:
print( monthInt_toStr('boom') )

In [ ]:
m_table = pd.Series( {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'June':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12} )
#m_strTable
print(m_table)

In [ ]:
a=[]
print(len(a))